In [0]:
# imports

import pandas as pd

In [0]:
# model output

PATH_DIR = '/content/drive/My Drive/'

df_prediction = pd.read_csv(PATH_DIR + 'als_predictions.csv')

df_prediction

,user,product,rating,true
0,1,307,4.359177,3.5
1,1,1590,2.821640,2.5
2,1,3424,4.613297,4.5
3,2,170,3.686878,3.5
4,2,1296,3.996778,4.5
...,...,...,...,...
5543343,283228,6874,4.176760,1.0
5543344,283228,7072,4.636644,0.5
5543345,283228,7121,4.603608,1.5
5543346,283228,7132,4.743483,2.0


In [0]:
# all movie ids vs titles

df_movies = \
  pd.read_csv(PATH_DIR + 'movies.csv', usecols=[0,1], names=['movieId', 'title'])

all_movies_dict = \
  pd.Series(df_movies['title'].values,index=df_movies['movieId']).to_dict()

In [0]:
# all users in test dataset 

all_users = df_prediction['user'].unique().tolist()

In [0]:
# results as per predicted ratings in test dataset 

df_prediction1 = df_prediction[['user', 'product', 'rating']]
df_prediction1['predicted_movies'] = df_prediction1 \
  .apply(lambda x: (x['product'], x['rating']), axis=1)

df_prediction2 = df_prediction1[['user', 'predicted_movies']]

df_prediction_formatted = df_prediction2 \
  .groupby('user')['predicted_movies'].apply(list).reset_index(name='recommendation')

df_prediction_sorted = df_prediction_formatted

df_prediction_sorted['recommendation'] = \
  df_prediction_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))
  
# df_prediction_sorted

sorted_reco_by_userid = pd.Series( \
  df_prediction_sorted['recommendation'].values,index=df_prediction_sorted['user']).to_dict()

In [0]:
def get_top_n_recommendations(user, n=10):
  top_n_reco = sorted_reco_by_userid[user][:n]
  
  return [all_movies_dict[str(int(x[0]))] for x in top_n_reco]

In [0]:
# results as per true ratings in test dataset

df_true1 = df_prediction[['user', 'product', 'true']]
df_true1['movies'] = df_true1 \
  .apply(lambda x: (x['product'], x['true']), axis=1)

df_true2 = df_true1[['user', 'movies']]

df_true_formatted = df_true2 \
  .groupby('user')['movies'].apply(list).reset_index(name='recommendation')

df_true_sorted = df_true_formatted

df_true_sorted['recommendation'] = \
  df_true_sorted['recommendation'].apply( \
    lambda x: sorted(x, key=lambda tup: tup[1], reverse=True))

# df_true_sorted

sorted_true_ratings_by_userid = pd.Series( \
  df_true_sorted['recommendation'].values,index=df_true_sorted['user']).to_dict()

In [0]:
def get_top_n_true_ratings(user, n=10):
  top_n_true_ratings = sorted_true_ratings_by_userid[user][:n]
  
  return [all_movies_dict[str(int(x[0]))] for x in top_n_true_ratings]

In [0]:
# output for each user in test dataset

top_n_recommendations_by_user = {}

user_out = []
for user in all_users:
  out = []

  # user
  out.append(user)

  # top n recommendations (n = 10 by default)
  top_n_reco_for_user = get_top_n_recommendations(user)
  top_n_recommendations_by_user[user] = top_n_reco_for_user

  top_n_true_ratings_for_user = get_top_n_true_ratings(user)
  true_positives = list(set(top_n_reco_for_user) & set(top_n_true_ratings_for_user))
  false_positives = list(set(top_n_reco_for_user) - set(top_n_true_ratings_for_user))
  false_negatives = list(set(top_n_true_ratings_for_user) - set(top_n_reco_for_user))

  # precision for user
  precision_for_user = len(true_positives) / float(len(true_positives) + len(false_positives))
  out.append(precision_for_user)

  # recall for user
  recall_for_user = len(true_positives) / float(len(true_positives) + len(false_negatives))
  out.append(recall_for_user)
  
  user_out.append(out)

In [0]:
# output for all users in test dataset

df_out = pd.DataFrame(user_out, columns=['userId', 'precision_user', 'recall_user'])
df_out

,userId,precision_user,recall_user
0,1,1.0,1.0
1,2,1.0,1.0
2,3,1.0,1.0
3,4,0.4,0.4
4,5,0.7,0.7
...,...,...,...
273646,283224,0.0,0.0
273647,283225,1.0,1.0
273648,283226,1.0,1.0
273649,283227,1.0,1.0


In [0]:
# top n recommendations for user 1

top_n_recommendations_by_user[1]

['Do the Right Thing (1989)',
 'Three Colors: Blue (Trois couleurs: Bleu) (1993)',
 'Event Horizon (1997)']

In [0]:
# EVALUATION

# mae
from sklearn.metrics import mean_absolute_error
mae = mean_absolute_error(df_prediction['true'], df_prediction['rating'])
print('mae', mae)

# rmse
from sklearn.metrics import mean_squared_error
from math import sqrt
rmse = sqrt(mean_squared_error(df_prediction['true'], df_prediction['rating']))
print('rmse', rmse)

# precision
precision = df_out['precision_user'].mean()
print('avg precision', precision)

# recall
recall = df_out['recall_user'].mean()
print('avg recall', recall)

# f measure
f_measure = (2.0 * precision * recall) / (precision + recall)
print('f measure', f_measure)

mae 1.0180018945558147
rmse 1.2938448110906982
avg precision 0.7779883866677325
avg recall 0.7779884272709502
f measure 0.7779884069693408
